In [0]:
from messages import Price, Trade
#import json
import queue
from statistics import stdev

In [14]:
packet = "TYPE=TRADE|FEEDCODE=SP-FUTURE|SIDE=ASK|PRICE=533.3|VOLUME=122"

trade = Trade.from_packet(packet)
print(trade.side)

ASK


In [0]:
# 2 queues of length 60
queue_ESX = queue.Queue(60)
queue_SP = queue.Queue(60)

d = dict()
d["feedcode"] = ""
d["price"] = 0
d["volume"] = 0

# hacky way to get around collection
collect_ESX = False
collect_SP = False

# If price is ESX  
if trade.feedcode == "ESX-FUTURE":
  # Fill until we reach the first 60
  if not queue_ESX.full() and not collect_ESX:
    queue_ESX.put(trade)
  else:
    collect_ESX = True
    queue_ESX.get(0)
    queue_ESX.put(trade)
    # call buyOrSell      
    d["feedcode"] = "ESX-FUTURE"
    d = buyOrSell(d,queue_ESX,trade.price)


# If price is SP 
else:
  if not queue_SP.full() and not collect_SP:
    queue_SP.put(trade)
  else: 
    collect_SP = True
    queue_SP.get(0)
    queue_SP.put(trade)
    d["feedcode"] = "SP-FUTURE"
    d = buyOrSell(d,queue,trade.price)
      

def buyOrSell(d,prev_60_val, newval):
  mean = sum(prev_60_val) / 60
  # calculate mean based on queue
  std = stdev(prev_60_val)
  # calculated std
  z = (newval - mean)/ std
  # calculate z-score
  # if z > 2 == negative value (sell)
  if z > 2:
    d["price"] = newval
    d["volume"] = -5
    return d
  # else if z < -2 == positive value (buy)
  elif z < -2:
    d["price"] = newval
    d["volume"] = 7
    return d
  
  else:
    # return 0 price, 0 volume
    return d


In [0]:
""